In [1]:
from astropy.io import fits
import astropy.wcs as wcs
import astropy.units as u
from astropy.convolution import convolve_fft
from astropy.stats import mad_std
import numpy as np
from radio_beam import Beam
from reproject import reproject_interp
from matplotlib import pyplot as plt
from scipy.ndimage import binary_dilation
from astropy.modeling import models, fitting
from astropy.table import Table, vstack 
from glob import glob 
import os 

import warnings 
warnings.filterwarnings('ignore')

In [2]:
rootdir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/'

# hdu_muse_f555w  = get_hdu(rootdir, 'muse/%s*_ACS_*F555W.fits' %galaxy_muse.upper())
# hdu_muse_f658n  = get_hdu(rootdir, 'muse/%s*_ACS_*F658N.fits' %galaxy_muse.upper())
# hdu_muse_f814w  = get_hdu(rootdir, 'muse/%s*_ACS_*F814W.fits' %galaxy_muse.upper())
# hdu_muse_stars  = get_hdu(rootdir, 'muse/%s_starmask.fits' %galaxy_muse.upper())
# hdu_muse_neb    = get_hdu(rootdir, 'muse/%s_nebmask.fits' %galaxy_muse.upper())
# hdu_muse        = get_hdu(rootdir, 'muse/%s*_MAPS.fits' %galaxy_muse.upper(), 'all')

# hdu_hst_f555w   = get_hdu(rootdir, 'hst/%s*_acs_*f555w*.fits' %galaxy)
# hdu_hst_f658n   = get_hdu(rootdir, 'hst/%s*_acs_*f658n*.fits' %galaxy)
# hdu_hst_f814w   = get_hdu(rootdir, 'hst/%s*_acs_*f814w*.fits' %galaxy)

In [3]:
os.walk('/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/.fits')

<generator object _walk at 0x7f8e82abc890>

In [5]:
import os

input_filenames = []
tables = [] 
headers = [] 

for root, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith("f657n_exp_drc_sci.fits") or file.endswith("f658n_exp_drc_sci.fits"):
            print(os.path.join(root, file))
            input_filenames += [os.path.join(root, file)]
            input_filename = os.path.join(root, file)

            header = fits.getheader(input_filename)
            header.tofile('headers/%s' %(file.replace('.fits', 'header.txt')), overwrite=True)
            headers += [header]
        
            
            name = input_filename.split('/')[-1].split('_exp')[0]
            galaxy = name.split('_')[0]
            instrument = name.split('_')[1]
            filter = name.split('_')[2]
            bw = header['PHOTBW']
            plam = header['PHOTPLAM']
            flam = header['PHOTFLAM']

            table = Table(np.array([galaxy, instrument, filter, np.float32(flam), np.float32(plam), np.float32(bw)]), 
                          names=['galaxy', 'instrument', 'filter', 'flam', 'plam', 'bw'])
            tables += [table]
        
fit_table_all = vstack(tables)
fit_table_all.sort('galaxy')

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/ngc4254/hst/ngc4254e_uvis_f657n_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/ngc1433/hst/ngc1433_uvis_f657n_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/ngc5068/hst/ngc5068n_uvis_f658n_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/ngc5068/hst/ngc5068s_uvis_f658n_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/ngc4321/hst/ngc4321_uvis_f657n_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/ngc1087/hst/ngc1087_uvis_f657n_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/ngc7496/hst/ngc7496_uvis_f657n_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/ngc1385/hst/ngc1385_uvis_f657n_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/ngc628e/hst/ngc628e_uvis_f658n_exp_drc_sci.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data/ngc33

In [ ]:
fit_table_all

galaxy,instrument,filter,flam,plam,bw
str32,str32,str32,str32,str32,str32
ngc1087,uvis,f657n,2.1906811e-18,6566.6133,41.036564
ngc1300,acs,f658n,1.9499036e-18,6583.951,37.010143
ngc1365n,uvis,f657n,2.1909498e-18,6566.6133,41.036564
ngc1385,uvis,f657n,2.190884e-18,6566.6133,41.036564
ngc1433,uvis,f657n,2.1856569e-18,6566.6133,41.036564
ngc1566,uvis,f658n,9.789709e-18,6583.9736,149.92953
ngc1672,acs,f658n,1.9531033e-18,6583.9517,37.014015
ngc3351,uvis,f658n,9.789046e-18,6583.9736,149.92953
ngc3351,uvis,f657n,2.1576e-18,6566.6,41.05


In [6]:
header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 7400                                                  
NAXIS2  =                 7400                                                  
EXTEND  =                    T                                                  
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
FILENAME= 'ngc1566_uvis_f658n_exp_drc_sci.fits' / name of file                  
                                                                                
FILETYPE= 'SCI      '          / Type of data in array                          
TELESCOP= 'HST'                / telescope used to acquire data                 
INSTRUME= 'WFC3  '          